In [129]:
import datetime 

import numpy as np
from numpy import *
#1.在区间范围内得到一个整数 2.当数值太大的时候进行调整
def loadDataSet(filename):
    fr = open(filename)
    dataMat = [];labelMat = []
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i#不想让i等于j
    while(j==i):
        j = int(random.uniform(0,m))
    return j

#对于大于H和小于L 的值进行调整
def clipAlpha(aj,H,L):
    if aj > H:
        aj=H
    if aj < L:
        aj = L
    return aj

def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    iter = 0
    m,n=shape(dataMatrix)
    b=0
    alphas = mat(zeros((m,1)))#取初值α0为0
    while(iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])#预测值和真实值的差
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b#随机选择第二个α
                Ej = fXj - float(labelMat[j])

                alphaIold = alphas[i].copy();
                alphaJold = alphas[j].copy();
                #当满足不等式约束 0<α<C的时候
                if (labelMat[i] != labelMat[j]):
                    L = max(0,alphas[j] - alphas[i])
                    H = min(C,C+alphas[j]- alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print("L==H");continue
                eta = 2*dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:].T
                if eta>=0:print("eta>=0");continue
                alphas[j] -= labelMat[j]*(Ei-Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)#将第二个参数进行剪切满足约束

                if (abs(alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold  - alphas[j])#得到第一个参数
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[i,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T

                if (0 < alphas[i]) and (C > alphas[i]): b = b1#为了符合KKT条件设定的，如果都不满足就用两者的中间数
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print("iteration number: %d" % iter)
    return b,alphas

class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler):#通过初始化创建一个数据结构对于数据进行存储，最好使用 python的字典进行处理，但是这为了方便，使用类似于java中的静态变量一样
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b =0
        self.eCache = mat(zeros((self.m,2)))#用于误差缓存
        
    def calcEk(oS,k):#得到Ek
        fXk = float(multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
        Ek = fXk - float(oS.labelMat[k])
        return Ek 
    #内循环中的启发式方法
    def selectJ(i,oS,Ei):
        maxK = -1; maxDeltaE = 0; Ej = 0
        oS.eCache[i] = [1,Ei]
        validEcacheList = nonzero(oS.eCache[:,0].A)[0]#构建一个非零表
        if (len(validEcacheList)) > 1:
            for k in validEcacheList:
                if k == i:
                    Ek = optStruct.calcEk(oS,k)
                    deltaE = abs(Ei-Ek)
                    if (deltaE > maxDeltaE):
                        maxK = k;maxDeltaE = deltaE;Ej = Ek#选择具有最大步长的j
            return maxK,Ej
        else:
            j = selectJrand(i,oS.m)
            Ej = optStruct.calcEk(oS,j)
        return j ,Ej
    
    def updateEk(oS,k):#计算误差并存入缓存
        Ek = optStruct.calcEk(oS,k)
        oS.eCache[k] = [1,Ek]
      
    #内循环代码
    def innerL(i,oS):
        Ei = optStruct.calcEk(oS, i)
        if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
            j,Ej = optStruct.selectJ(i, oS, Ei) #selectJrand变成了selectJ在选择第二个变量的时候进行优化
            alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
            if (oS.labelMat[i] != oS.labelMat[j]):
                L = max(0, oS.alphas[j] - oS.alphas[i])
                H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
            else:
                L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
                H = min(oS.C, oS.alphas[j] + oS.alphas[i])
            if L==H: print("L==H"); return 0
            eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T- oS.X[j,:]*oS.X[j,:].T
            #eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
            if eta >= 0: print("eta>=0"); return 0
            oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
            oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
            optStruct.updateEk(oS, j) #误差加入到Ecache
            if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j不满足"); return 0
            oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
            optStruct.updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
            b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
            b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
            if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
            elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
            else: oS.b = (b1 + b2)/2.0
            return 1
        else: return 0
     
    #外循环代码,相对于simpleSmo来说while的循环退出条件更多
    def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
        oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler)
        iter = 0
        entireSet = True; alphaPairsChanged = 0
        while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):#都未作出修改且小于最大循环数
            alphaPairsChanged = 0
            if entireSet:
                for i in range(oS.m):
                    alphaPairsChanged += optStruct.innerL(i,oS)
                print("")
                print("")
                print("整个set,迭代: %d 第: %d alpha改变 %d " % (iter,i,alphaPairsChanged))
                iter += 1
            else:#遍历非边界
                nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
                for i in nonBoundIs:
                    alphaPairsChanged += optStruct.innerL(i,oS)
                    print("")
                    print("")
                    print("不在边界上, 迭代: %d 第:%d, alpha改变 %d" % (iter,i,alphaPairsChanged))
                iter += 1
            if entireSet: entireSet = False
            elif (alphaPairsChanged == 0):entireSet = True
            print("迭代数: %d" % iter)
        return oS.b,oS.alphas
                    
def calcWs(alphas,dataArr,classLabels):
    X = mat(dataArr);labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

def kernelTrans(X,A,kTup):
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0] == 'lin':K = X*A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] -A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2))
    else: raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K

class optStructNew:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
        
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]
        
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print("L==H"); return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j不满足"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):    #full Platt SMO
    oS = optStructNew(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print("整个set,迭代: %d 第: %d alpha改变 %d " % (iter,i,alphaPairsChanged))
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("不在边界上, 迭代: %d 第:%d, alpha改变 %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print("迭代数: %d" % iter)
    return oS.b,oS.alphas

def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch06/testSetRBF.txt')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,10000,('rbf',k1))
    dataMat = mat(dataArr) ;labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]#找出非0的alpha值
    sVs =dataMat[svInd]#得到支持向量
    labelSV = labelMat[svInd]#构建支持向量矩阵
    print("这里有 %d 支持向量" % shape(sVs)[0])
    m,n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]) : errorCount += 1
    print("训练集的错误率为: %f " % (float(errorCount)/m))
    
    dataArr,labelArr = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch06/testSetRBF2.txt')
    errorCount = 0
    dataMat = mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]) : errorCount += 1
    print("测试集的错误率为: %f " % (float(errorCount)/m))
    
    
    
    
#识别手写数字

def image2vector(filename):
    vector = zeros((1,1024))
    
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            vector[0][32*i+j] = int(lineStr[j])
    return vector

from os import listdir
def loadImages(dirName):
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = image2vector('%s/%s' % (dirName,fileNameStr))
    return trainingMat,hwLabels

def testDigits(kTup=('rbf',10)):
    dataArr,labelArr = loadImages('E:谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch06/digits/trainingDigits')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,10000,kTup)
    dataMat = mat(dataArr);labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]
    sVs = dataMat[svInd]
    labelSV = labelMat[svInd]
    print("有 %d 个支持向量" % shape(sVs)[0])
    m,n = shape(dataMat)
    
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict = kernelEval.T*multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):errorCount += 1
    print("训练的错误率为 %f" % (float(errorCount/m)))
    
    dataArr,labelArr = loadImages('E:谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch06/digits/testDigits')
    errorCount = 0
    dataMat = mat(dataArr);labelMat = mat(labelArr).transpose()
    m,n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict = kernelEval.T*multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):errorCount += 1
    print("测试的错误率为 %f" % (float(errorCount/m)))

In [92]:
dataArr,labelArr = loadDataSet('E:/谢远东/机器学习/机器学习实践/机器学习实战源代码/machinelearninginaction/Ch06/testSet.txt')

In [93]:
dataArr

[[3.542485, 1.977398],
 [3.018896, 2.556416],
 [7.55151, -1.58003],
 [2.114999, -0.004466],
 [8.127113, 1.274372],
 [7.108772, -0.986906],
 [8.610639, 2.046708],
 [2.326297, 0.265213],
 [3.634009, 1.730537],
 [0.341367, -0.894998],
 [3.125951, 0.293251],
 [2.123252, -0.783563],
 [0.887835, -2.797792],
 [7.139979, -2.329896],
 [1.696414, -1.212496],
 [8.117032, 0.623493],
 [8.497162, -0.266649],
 [4.658191, 3.507396],
 [8.197181, 1.545132],
 [1.208047, 0.2131],
 [1.928486, -0.32187],
 [2.175808, -0.014527],
 [7.886608, 0.461755],
 [3.223038, -0.552392],
 [3.628502, 2.190585],
 [7.40786, -0.121961],
 [7.286357, 0.251077],
 [2.301095, -0.533988],
 [-0.232542, -0.54769],
 [3.457096, -0.082216],
 [3.023938, -0.057392],
 [8.015003, 0.885325],
 [8.991748, 0.923154],
 [7.916831, -1.781735],
 [7.616862, -0.217958],
 [2.450939, 0.744967],
 [7.270337, -2.507834],
 [1.749721, -0.961902],
 [1.803111, -0.176349],
 [8.804461, 3.044301],
 [1.231257, -0.568573],
 [2.074915, 1.41055],
 [-0.743036, -1.73

In [94]:
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [95]:
starttime = datetime.datetime.now() #long running #do something other 
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)
b


iter: 0 i:0, pairs changed 1
j not moving enough
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
iter: 0 i:18, pairs changed 2
L==H
L==H
j not moving enough
iter: 0 i:29, pairs changed 3
L==H
iter: 0 i:54, pairs changed 4
L==H
L==H
L==H
iter: 0 i:81, pairs changed 5
L==H
j not moving enough
L==H
iter: 0 i:94, pairs changed 6
j not moving enough
iteration number: 0
iter: 0 i:0, pairs changed 1
L==H
j not moving enough
iter: 0 i:5, pairs changed 2
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
iter: 0 i:26, pairs changed 3
j not moving enough
L==H
iter: 0 i:52, pairs changed 4
iter: 0 i:54, pairs changed 5
j not moving enough
L==H
j not moving enough
iter: 0 i:94, pairs changed 6
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
iter: 0 i:5, pairs changed 1
j not moving enough
j not moving enough
iter: 0 i:17, pairs changed 2
iter: 0 i:18, pairs changed 3
j not moving enough
j not moving enough
j not mov

matrix([[-3.8215337]])

In [96]:
alphas[alphas>0]

matrix([[ 0.09560467,  0.26418153,  0.02305629,  0.3367299 ]])

In [97]:
m,n = shape(alphas[alphas>0])
for i in range(n):
    if alphas[i] >0.0:print(dataArr[i],labelArr[i])
endtime = datetime.datetime.now() 

In [98]:
print(endtime - starttime)

0:00:02.784975


In [99]:
starttime = datetime.datetime.now() #long running #do something other 
b,alphas = optStruct.smoP(dataArr,labelArr,0.6,0.001,40)
endtime = datetime.datetime.now() 
print(endtime - starttime)

L==H
L==H
eta>=0


整个set,迭代: 0 第: 99 alpha改变 18 
迭代数: 1


不在边界上, 迭代: 1 第:2, alpha改变 1


不在边界上, 迭代: 1 第:3, alpha改变 2


不在边界上, 迭代: 1 第:4, alpha改变 3


不在边界上, 迭代: 1 第:5, alpha改变 4


不在边界上, 迭代: 1 第:7, alpha改变 5


不在边界上, 迭代: 1 第:8, alpha改变 6


不在边界上, 迭代: 1 第:13, alpha改变 7


不在边界上, 迭代: 1 第:17, alpha改变 8


不在边界上, 迭代: 1 第:18, alpha改变 9


不在边界上, 迭代: 1 第:22, alpha改变 10


不在边界上, 迭代: 1 第:25, alpha改变 11


不在边界上, 迭代: 1 第:26, alpha改变 12


不在边界上, 迭代: 1 第:29, alpha改变 13


不在边界上, 迭代: 1 第:45, alpha改变 14


不在边界上, 迭代: 1 第:46, alpha改变 15


不在边界上, 迭代: 1 第:54, alpha改变 16


不在边界上, 迭代: 1 第:55, alpha改变 17


不在边界上, 迭代: 1 第:97, alpha改变 18
eta>=0


不在边界上, 迭代: 1 第:99, alpha改变 18
迭代数: 2


不在边界上, 迭代: 2 第:2, alpha改变 1


不在边界上, 迭代: 2 第:4, alpha改变 2


不在边界上, 迭代: 2 第:5, alpha改变 3


不在边界上, 迭代: 2 第:13, alpha改变 4


不在边界上, 迭代: 2 第:18, alpha改变 5


不在边界上, 迭代: 2 第:22, alpha改变 6


不在边界上, 迭代: 2 第:25, alpha改变 7


不在边界上, 迭代: 2 第:26, alpha改变 8
j不满足


不在边界上, 迭代: 2 第:45, alpha改变 8


不在边界上, 迭代: 2 第:46, alpha改变 9


不在边界上, 迭代: 2 第:54, alph

In [100]:
b

matrix([[-4.22736871]])

In [101]:
alphas

matrix([[  0.00000000e+00],
        [  0.00000000e+00],
        [ -6.93889390e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  6.93889390e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -1.73472348e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -1.73472348e-18],
        [ -3.46944695e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -6.93889390e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -8.67361738e-18],
        [ -1.38777878e-17],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [ -6.93889390e-18],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        [  0.0000000

In [102]:
ws = calcWs(alphas,dataArr,labelArr)
ws

array([[ 0.878598  ],
       [-0.27456292]])

In [103]:
dataMat = mat(dataArr)
dataMat[0]*ws + b


matrix([[-1.65786864]])

In [104]:
labelArr[0]

-1.0

In [105]:
dataMat[2]*ws +b

matrix([[ 2.84119051]])

In [106]:
labelArr[2]

1.0

In [109]:
testRbf()

L==H
整个set,迭代: 0 第: 0 alpha改变 0 
整个set,迭代: 0 第: 1 alpha改变 1 
整个set,迭代: 0 第: 2 alpha改变 2 
整个set,迭代: 0 第: 3 alpha改变 3 
整个set,迭代: 0 第: 4 alpha改变 3 
整个set,迭代: 0 第: 5 alpha改变 4 
整个set,迭代: 0 第: 6 alpha改变 5 
整个set,迭代: 0 第: 7 alpha改变 5 
整个set,迭代: 0 第: 8 alpha改变 6 
整个set,迭代: 0 第: 9 alpha改变 6 
整个set,迭代: 0 第: 10 alpha改变 7 
整个set,迭代: 0 第: 11 alpha改变 8 
j不满足
整个set,迭代: 0 第: 12 alpha改变 8 
整个set,迭代: 0 第: 13 alpha改变 9 
整个set,迭代: 0 第: 14 alpha改变 10 
整个set,迭代: 0 第: 15 alpha改变 11 
整个set,迭代: 0 第: 16 alpha改变 12 
整个set,迭代: 0 第: 17 alpha改变 12 
整个set,迭代: 0 第: 18 alpha改变 13 
整个set,迭代: 0 第: 19 alpha改变 14 
整个set,迭代: 0 第: 20 alpha改变 14 
整个set,迭代: 0 第: 21 alpha改变 15 
整个set,迭代: 0 第: 22 alpha改变 15 
整个set,迭代: 0 第: 23 alpha改变 15 
整个set,迭代: 0 第: 24 alpha改变 16 
j不满足
整个set,迭代: 0 第: 25 alpha改变 16 
整个set,迭代: 0 第: 26 alpha改变 16 
j不满足
整个set,迭代: 0 第: 27 alpha改变 16 
整个set,迭代: 0 第: 28 alpha改变 16 
整个set,迭代: 0 第: 29 alpha改变 16 
j不满足
整个set,迭代: 0 第: 30 alpha改变 16 
整个set,迭代: 0 第: 31 alpha改变 17 
整个set,迭代: 0 第: 32 alpha改变 17 
整个set,迭代:

In [130]:
testDigits()

整个set,迭代: 0 第: 0 alpha改变 1 
整个set,迭代: 0 第: 1 alpha改变 2 
整个set,迭代: 0 第: 2 alpha改变 3 
整个set,迭代: 0 第: 3 alpha改变 4 
整个set,迭代: 0 第: 4 alpha改变 5 
整个set,迭代: 0 第: 5 alpha改变 6 
整个set,迭代: 0 第: 6 alpha改变 7 
整个set,迭代: 0 第: 7 alpha改变 8 
整个set,迭代: 0 第: 8 alpha改变 9 
整个set,迭代: 0 第: 9 alpha改变 10 
整个set,迭代: 0 第: 10 alpha改变 11 
整个set,迭代: 0 第: 11 alpha改变 12 
整个set,迭代: 0 第: 12 alpha改变 13 
整个set,迭代: 0 第: 13 alpha改变 14 
整个set,迭代: 0 第: 14 alpha改变 15 
整个set,迭代: 0 第: 15 alpha改变 16 
整个set,迭代: 0 第: 16 alpha改变 17 
整个set,迭代: 0 第: 17 alpha改变 18 
整个set,迭代: 0 第: 18 alpha改变 19 
整个set,迭代: 0 第: 19 alpha改变 20 
整个set,迭代: 0 第: 20 alpha改变 21 
整个set,迭代: 0 第: 21 alpha改变 22 
整个set,迭代: 0 第: 22 alpha改变 23 
整个set,迭代: 0 第: 23 alpha改变 24 
整个set,迭代: 0 第: 24 alpha改变 25 
整个set,迭代: 0 第: 25 alpha改变 26 
整个set,迭代: 0 第: 26 alpha改变 26 
整个set,迭代: 0 第: 27 alpha改变 26 
整个set,迭代: 0 第: 28 alpha改变 26 
整个set,迭代: 0 第: 29 alpha改变 27 
整个set,迭代: 0 第: 30 alpha改变 28 
整个set,迭代: 0 第: 31 alpha改变 28 
整个set,迭代: 0 第: 32 alpha改变 29 
整个set,迭代: 0 第: 33 alpha改变 29 